In [3]:
%reload_ext autoreload
%autoreload 1
%aimport train

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [5]:
import pickle
with open('vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)
    vocab = np.array(vocab)

In [9]:
doc_data = pd.read_pickle('data_tokenized.pkl')

In [10]:
trainer, X1, X2, Y = train.build_nce_model(num_words = len(vocab), num_docs = len(doc_data))

In [30]:
trainer.restore('embedding_model_nce-51230')

In [31]:
e1, = trainer.graph.get_collection('layer_variables/Embedding/')
e2, = trainer.graph.get_collection('layer_variables/Embedding_1/')

doc_embedding = e1.eval(session=trainer.session)
word_embedding = e2.eval(session=trainer.session)

In [32]:
doc_embedding_series = pd.Series(list(doc_embedding))

In [33]:
doc_embedding_norm = doc_embedding / np.linalg.norm(doc_embedding, axis=1, keepdims=True)

In [43]:
news_data = doc_data[doc_data.document_kind == 'news']

In [59]:
news_embedding = doc_embedding_norm[news_data.index]

In [60]:
doc_id = 620
doc = doc_embedding_norm[doc_id]
print(doc_data.iloc[doc_id].text)

dist = np.sum(doc * news_embedding, axis=1)
dist_sort_idx = np.argsort(dist)[::-1]
for i in dist_sort_idx[:25]:
    t = news_data.iloc[i].text
    d = dist[i]
    print('---', np.round(d, 2))
    print(t)

Aus real GDP grew 0.6% in Dec quarter &amp; by 3.0% compared to a year ago, up 2.7% through the year to Sept #nationalaccounts #auspol
--- 0.59
Election FactCheck: Has the Coalition presided over the most sustained fall in Australian living standards since records began?
Editions
Africa
Australia
France
United Kingdom
United States
Sections
Home
Arts + Culture
Business + Economy
Education
Environment + Energy
Health + Medicine
Politics + Society
Science + Technology
Search
Services
Events
Job board
Newsletter
Read on Google Play
Information
Who we are
Our charter
Our team
Our audience
Partners and funders
Contributing institutions
Contact us
Friends of The Conversation
Privacy policy
Terms and conditions
Corrections
Edition:
Australia
Africa
France
United Kingdom
United States
Job Board
Donate
Become an author
Sign up as a reader
Sign in
Get newsletter
Search
Academic rigour, journalistic flair
Arts + Culture
Business + Economy
Education
Environment + Energy
Health + Medicine
Politics 

In [35]:
doc = doc_embedding_norm[202]

dist = np.sum(doc * doc_embedding_norm, axis=1)
dist_sort_idx = np.argsort(dist)[::-1]
for i in dist_sort_idx[:25]:
    t = doc_data.iloc[i].text
    d = dist[i]
    print('---', np.round(d, 2))
    print(t)

--- 1.0
Always great to be talking on @RTRFM https://t.co/7vC3gqfi4f
--- 0.69
Shortly I'll be talking to Sally Warhaft on @774melbourne about #auspol https://t.co/ziMM1J2RJ5
--- 0.63
Looking forward to talking with Paul Murray on @pm_live shortly
--- 0.63
I'm talking to @jimmiddleton on @SkyNewsAust from Melbourne #auspol #ausvotes
--- 0.62
I'll be joining @GreenJ on @RNDrive shortly to chat #auspol #ausvotes  https://t.co/ZWBEfOW9CI
--- 0.62
I'm talking to @jimmiddleton now on @SkyNewsAust about #auspol #ausvotes
--- 0.59
Appearing on #tothepoint on @SkyNewsAust with @KKeneally &amp; @ash_gillon https://t.co/poqttA2uA4
--- 0.58
I'll be joining @Kieran_Gilbert for #amagenda on @SkyNewsAust shortly #auspol
--- 0.57
Will work on that
--- 0.57
I'll be speaking with @PatsKarvelas on @SkyNewsAust shortly
--- 0.57
I'll be speaking with #RayHadley this morning at 9.20am on @2GB873 #auspol
--- 0.56
@niall_tangney Will be there on 19th!
--- 0.56
Congrats on strategy! https://t.co/mYMQIpPx0D
---

In [36]:
word_embedding

array([[ 0.02643727,  0.12004966, -0.08888668, ...,  0.03794339,
        -0.01791329, -0.00073096],
       [-0.33643791,  0.15888341, -0.13873135, ..., -0.00517062,
         0.02794992,  0.1951692 ],
       [-0.55893254, -0.19542348, -0.15763251, ..., -0.46800411,
         0.05891257, -0.25182819],
       ..., 
       [-0.01308417,  0.34089622, -0.47103262, ...,  0.23765527,
         0.2978816 ,  0.49335513],
       [ 0.00686084,  0.23487678, -0.30312827, ...,  0.33598581,
         0.53408527,  0.55388713],
       [ 0.03378136, -0.03530313, -0.05638266, ...,  0.10530957,
         0.06166964,  0.00986569]], dtype=float32)

In [37]:
word_embedding_norm = word_embedding / np.linalg.norm(word_embedding, axis=1, keepdims=True)

In [38]:
word_to_vocab = {w: i for i, w in enumerate(vocab)}

def e(s):
    return word_embedding[word_to_vocab[s]]

In [39]:
# word = e('Turnbull') - e('Liberal') + e('Labor')
# word = e('Shorten') - e('Labor') + e('Liberal')

# word = e('plebiscite') - e('gay') - e('straight')

word = e('Liberal')
# word = e('Labor')
# word = e('Greens')
# word = e('plebiscite')
# word = e('gay')
# word = e('sexism')
# word = e('budget')
# word = e('tax')
# word = e('debt')
# word = e('unemployment')
# word = e('foreign')
# word = e('TPP')
# word = e('anger')
# word = e('pollution')

word_norm = word / np.linalg.norm(word)
dist = np.sum(word_norm * word_embedding_norm, axis=1)
dist_sort_idx = np.argsort(dist)[::-1]
for i in dist_sort_idx[:25]:
    t = vocab[i]
    d = dist[i]
    print(np.round(d, 2), t)

1.0 Liberal
0.73 Lib
0.62 Nationals
0.58 LNP
0.57 @The_Nationals
0.53 Lazarus
0.49 QLD
0.49 Labor
0.48 National
0.48 Fed
0.45 energy
0.45 Search
0.44 @LiberalAus
0.44 George
0.44 @AustralianLabor
0.44 Glenn
0.44 disaster
0.44 federal
0.43 Victorian
0.43 Tasmanian
0.42 Christmas
0.42 @AlanTudgeMP
0.42 Macquarie
0.42 #
0.41 Adam
